# Redes Neurais Convolucionais

Neste notebook, exploraremos os fundamentos das Redes Neurais Convolucionais (Convolutional Neural Networks, ou CNNs), uma classe de redes neurais profundas que se tornou dominante em diversas tarefas de visão computacional.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets
from torchvision.transforms import ToTensor
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

### Preparação dos Dados e do Treinamento

In [ ]:
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

validation_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_subset = Subset(training_data, range(5000))
validation_subset = Subset(validation_data, range(1000))

batch_size = 64
train_dataloader = DataLoader(train_subset, batch_size=batch_size)
validation_dataloader = DataLoader(validation_subset, batch_size=batch_size)

In [ ]:
def train_model(model, train_dataloader, validation_dataloader, criterion, optimizer, epochs=10):
    history = {
        'train_loss': [],
        'train_acc': [],
        'val_loss': [],
        'val_acc': []
    }

    model.to(device)

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / total_samples
        epoch_acc = correct_predictions / total_samples
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc)

        model.eval()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        with torch.no_grad():
            for inputs, labels in validation_dataloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total_samples += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

        epoch_val_loss = running_loss / total_samples
        epoch_val_acc = correct_predictions / total_samples
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)

        print(f"Epoch {epoch+1}/{epochs} - "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f} - "
              f"Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.4f}")

    return history

def plot_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    ax1.plot(history['train_acc'])
    ax1.plot(history['val_acc'])
    ax1.set_title('Model accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend(['Train', 'Validation'], loc='upper left')
    ax1.grid(True)

    ax2.plot(history['train_loss'])
    ax2.plot(history['val_loss'])
    ax2.set_title('Model loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend(['Train', 'Validation'], loc='upper left')
    ax2.grid(True)
    
    plt.show()

### As Camadas de uma CNN

Redes Neurais Convolucionais introduzem camadas especializadas que são projetadas para extrair características hierárquicas de dados com estrutura de grade, como imagens. As duas camadas mais fundamentais são a convolucional (`Conv2d`) e a de pooling (`MaxPool2d`).

#### Convolução 2D (`Conv2d`)

A camada convolucional é o principal bloco de construção de uma CNN. Sua função é aplicar um conjunto de filtros (ou kernels) à imagem de entrada. Cada filtro é uma pequena matriz de pesos que desliza sobre a imagem, computando o produto de ponto entre os pesos do filtro e a porção da imagem sob ele. Este processo gera um mapa de ativação (ou mapa de características) que indica a presença de uma característica específica (e.g., uma borda vertical, uma curva) em diferentes locais da imagem.

Os principais parâmetros da camada `torch.nn.Conv2d` são:
* `in_channels`: O número de canais da imagem de entrada (e.g., 1 para escala de cinza, 3 para RGB).
* `out_channels`: O número de filtros a serem aplicados. Cada filtro produzirá um canal no mapa de características de saída.
* `kernel_size`: As dimensões (altura x largura) do filtro.
* `stride`: O passo com que o filtro se move pela imagem. Um `stride` de 1 significa que o filtro se move um pixel de cada vez.
* `padding`: Adição de pixels (geralmente com valor zero) ao redor da imagem de entrada. Isso é útil para controlar a dimensão espacial da saída.

A dimensão da saída de uma camada convolucional pode ser calculada pela seguinte fórmula:

$$
W_{out} = \frac{W_{in} - K + 2P}{S} + 1
$$

Onde:
* $W_{in}$ é a largura da entrada.
* $K$ é o tamanho do kernel.
* $P$ é o padding.
* $S$ é o stride.

A mesma fórmula se aplica à altura ($H_{out}$).

In [ ]:
sample_image = torch.randn(1, 1, 28, 28) # (N, C_in, H, W)

conv_layer = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)

feature_map = conv_layer(sample_image)

print(f"Input image: {sample_image.shape}")
print(f"Feature map: {feature_map.shape}")

#### Pooling (`MaxPool2d`)

A camada de pooling é comumente inserida entre camadas convolucionais sucessivas em uma arquitetura de CNN. Seu objetivo principal é reduzir progressivamente a dimensão espacial (altura e largura) da representação, o que diminui a quantidade de parâmetros e computação na rede. Isso também ajuda a tornar a representação mais robusta a pequenas translações na imagem de entrada (invariância à translação).

A operação de Max Pooling, implementada por `torch.nn.MaxPool2d`, seleciona o valor máximo de uma vizinhança definida pelo `kernel_size` no mapa de características.

Os principais parâmetros são:
* `kernel_size`: O tamanho da janela sobre a qual a operação de max pooling é aplicada.
* `stride`: O passo com que a janela se move. Frequentemente, o `stride` é igual ao `kernel_size` para evitar sobreposição.

In [ ]:
pool_layer = nn.MaxPool2d(kernel_size=2, stride=2)

pooled_output = pool_layer(feature_map)

print(f"Feature map: {feature_map.shape}")
print(f"Output: {pooled_output.shape}")

#### Convolução Transposta (`ConvTranspose2d`)

Enquanto a convolução e o pooling são operações que tipicamente reduzem a dimensionalidade espacial (downsampling) dos mapas de características, há cenários em que o processo inverso (upsampling) é necessário. A camada `ConvTranspose2d`, frequentemente, embora de forma imprecisa, chamada de "deconvolução", serve a este propósito.

Sua função é realizar um upsampling do mapa de características de entrada para uma resolução espacial maior. Em vez de mapear uma vizinhança de múltiplos valores de entrada para um único valor de saída (como na convolução padrão), a convolução transposta mapeia um único valor de entrada para uma vizinhança de múltiplos valores de saída. Ela realiza esta operação aprendendo um conjunto de filtros, similar à `Conv2d`, mas o cálculo é estruturado para expandir as dimensões de altura e largura.

Esta camada é fundamental em arquiteturas como autoencoders (na porção do decodificador), redes generativas (GANs) e em tarefas de segmentação semântica, onde é preciso reconstruir uma imagem ou um mapa de segmentação na resolução original a partir de um mapa de características de baixa dimensão.

A dimensão da saída de uma camada de convolução transposta pode ser calculada pela seguinte fórmula:

$$
W_{out} = (W_{in} - 1) \times S - 2P + K + OP
$$

Onde:
* $W_{in}$ é a largura da entrada.
* $S$ é o `stride`.
* $P$ é o `padding`.
* $K$ é o `kernel_size`.
* $OP$ é o `output_padding`, um parâmetro que ajuda a resolver ambiguidades no cálculo do tamanho da saída.

In [ ]:
transpose_conv_layer = nn.ConvTranspose2d(
    in_channels=16, 
    out_channels=8, 
    kernel_size=2, 
    stride=2
)

upsampled_output = transpose_conv_layer(pooled_output)

print(f"Feature map: {pooled_output.shape}")
print(f"Transposed convolution: {upsampled_output.shape}")

In [ ]:
input_tensor = torch.tensor([[[
    [0, 1],
    [2, 3]]
]])
kernel = torch.tensor([[[
    [0, 1],
    [2, 3]
]]])

output_tensor = nn.functional.conv_transpose2d(input_tensor, kernel, stride=1, padding=0)

print(f"Input:\n{input_tensor.squeeze()}\n")
print(f"Kernel:\n{kernel.squeeze()}\n")
print(f"Output:\n{output_tensor.squeeze()}")

### Construção de um Modelo CNN para Classificação

Agora, vamos combinar as camadas `Conv2d` e `MaxPool2d`, junto com uma função de ativação não-linear como a ReLU (`nn.ReLU`) e camadas lineares (`nn.Linear`) ao final, para construir uma CNN completa para a tarefa de classificação do MNIST.

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.flatten = nn.Flatten()
        self.linear_block = nn.Sequential(
            nn.Linear(32 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.flatten(x)
        logits = self.linear_block(x)
        return logits

model = SimpleCNN().to(device)
print(model)

### Treinamento e Avaliação do Modelo

Com o modelo definido, podemos agora instanciar a função de perda e o otimizador. Utilizaremos `CrossEntropyLoss` para a classificação multi-classe e o otimizador `Adam`. Em seguida, passaremos tudo para nossa função de treinamento.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

num_epochs = 10
history = train_model(
    model=model,
    train_dataloader=train_dataloader,
    validation_dataloader=validation_dataloader,
    criterion=loss_fn,
    optimizer=optimizer,
    epochs=num_epochs
)

In [ ]:
plot_history(history)

In [ ]:
import math

filters = model.cnn[0].weight.detach().cpu()

f_min, f_max = filters.min(), filters.max()
filters_normalized = (filters - f_min) / (f_max - f_min)

num_filters = filters_normalized.shape[0]
cols = int(math.ceil(math.sqrt(num_filters)))
rows = int(math.ceil(num_filters / cols))

fig, axes = plt.subplots(rows, cols, figsize=(8, 8))
fig.suptitle('Filtros da Primeira Camada Convolucional', fontsize=16)
axes = axes.flatten()

for i in range(num_filters):
    filt = filters_normalized[i, 0, :, :]

    ax = axes[i]
    ax.imshow(filt, cmap='gray')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Filtro {i+1}")

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()

## Exercícios

### Exercício 1

Altere o dataset e o modelo para verificar se o dígito é ímpar ou par. A saída deve contar apenas um neurônio com função de ativação sigmoid.

### Exercício 2

Faça um modelo que receba como entrada um vetor de formato `(N, latent_dim)` e tenha como saída um tensor representando uma imagem com formato `(N, 3, 100, 100)`, onde `N` é o batch size.